### 5. 
Refer to the Bleichenbacher's attack as mentioned in paper title "Chosen Ciphertext Attacks Against Protocols Based on the RSA Encryption Standard PKCS #1". Follow the underlying steps:


a. Generate RSA key pairs, where the public key encryption exponent “e” should be
3. And consider the message “m” to be your roll number (all small letters). For eg
- "p21cs011".
p and q should be 128 bit prime numbers.


In [2]:
m='b21cs050'

b. Encrypt the message “m” using the RSA algorithm with PKCS1.5 padding to
obtain ciphertext “x” (Inbuilt libraries support automatic message conversion
followed by encryption). You can use any cryptographic hashing function for the
same.


In [1]:
from Crypto.Cipher import PKCS1_v1_5
# pad my message
from Crypto.PublicKey import RSA
from Crypto.Hash import SHA




c. Design a server (or function) that accepts the ciphertext, decrypts it using the
private key and verifies the message (plaintext) has the first two bytes as 00h 01h.
The server returns a boolean value indicating the verification results. For instance
- If the first two bytes are anything other than 00h 01h, the server responds with
“False” value. On the other hand, the server returns “True” value.


Your task is to decrypt the ciphertext “x” using the server defined in this question
following the Bleichenbacher's attack.